In [47]:
import torch

In [48]:
class Dropout:
    def __init__(self, p=0.5):
        self.p = p
        self.parameter = {'name': 'Dropout', 'p': p}

    def forward(self, X):
        self.mask = (torch.rand_like(X) > self.p) / (1 - self.p)
        return X * self.mask

    def backward(self, d_y):
        return d_y * self.mask


In [49]:
class RMSNorm:
    def __init__(self, num_features, eps=1e-5):
        self.num_features = num_features
        self.eps = eps

        self.gamma = torch.ones(1, num_features)
        self.beta = torch.zeros(1, num_features)

        self.X, self.X_hat, self.rms, self.d_gamma, self.d_beta = None, None, None, None, None

        self.parameter = {'name': 'RMSNorm', 'size': [1, num_features]}

    def forward(self, X):
        self.X = X
        self.rms = torch.sqrt(X.pow(2).mean(dim=1, keepdim=True) + self.eps)
        self.X_hat = X / self.rms
        return self.X_hat * self.gamma + self.beta

    def backward(self, d_y):
        self.d_gamma = (d_y * self.X_hat).sum(dim=0, keepdim=True)
        self.d_beta = d_y.sum(dim=0, keepdim=True)
        
        d_X_hat = d_y * self.gamma
        inv_rms = 1 / self.rms
        d_X = inv_rms * d_X_hat - self.X * (self.X * d_X_hat).mean(dim=1, keepdim=True) * inv_rms ** 3
        return d_X

        

In [50]:
class LayerNorm1d:
    def __init__(self, num_features, eps=1e-5):
        self.num_features = num_features
        self.eps = eps

        self.gamma = torch.ones(1, num_features)
        self.beta = torch.zeros(1, num_features)

        self.X_hat, self.std_inv, self.d_gamma, self.d_beta = None, None, None, None

        self.parameter = {'name': 'LayerNorm1d', 'size': [1, num_features]}

    def forward(self, X):
        mean = X.mean(dim=1, keepdim=True)
        var = X.var(dim=1, unbiased=False, keepdim=True)
        std = torch.sqrt(var + self.eps)

        self.X_hat = (X - mean) / std
        self.std_inv = 1 / std

        y = self.gamma * self.X_hat + self.beta
        return y

    def backward(self, d_y):
        self.d_gamma = (d_y * self.X_hat).sum(dim=0, keepdim=True)
        self.d_beta = d_y.sum(dim=0, keepdim=True)

        d_X_hat = self.gamma * d_y
        mean_dxhat = d_X_hat.mean(dim=1, keepdim=True)
        mean_dxhat_xhat = (d_X_hat * self.X_hat).mean(dim=1, keepdim=True)
        d_X = (d_X_hat - mean_dxhat - self.X_hat * mean_dxhat_xhat) * self.std_inv

        return d_X

        

In [51]:
class BatchNorm1d:
    def __init__(self, num_features, eps=1e-5, momentum=0.9):
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum

        self.gamma = torch.ones(1, num_features)       # (1, C)
        self.beta  = torch.zeros(1, num_features)      # (1, C)
        self.running_mean = torch.zeros(1, num_features)
        self.running_var  = torch.ones(1, num_features)

        # caches
        self.X_hat, self.std_inv, self.m, self.d_gamma, self.d_beta = None, None, None, None, None

        self.parameter = {'name': 'BatchNorm1d', 'size': [1, num_features]}

    def forward(self, X):
        mean = X.mean(dim=0, keepdim=True)
        var = X.var(dim=0, unbiased=False, keepdim=True)
        std = torch.sqrt(var + self.eps)

        self.X_hat = (X - mean) / std
        self.std_inv = 1 / std
        self.m = X.shape[0]

        self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * mean
        self.running_var = self.momentum * self.running_var + (1 - self.momentum) * var

        y = self.gamma * self.X_hat + self.beta
        return y

    def backward(self, d_y): 
        # y = gamma * X_hat + beta (B, C), X_hat (B, C)
        # X_hat = (X - mean) / std
        m = self.m
        self.d_gamma = (d_y * self.X_hat).sum(dim=0, keepdim=True) # (1, C), Hadamard product, sum each batch's contribution
        self.d_beta = d_y.sum(dim=0, keepdim=True) # (1, C)
        dX = (1.0 / m) * self.gamma * self.std_inv * (m * d_y - self.d_beta - self.X_hat * self.d_gamma)
        return dX
        
            

In [52]:
class LinearLayer:
    def __init__(self, input_dim, out_dim, batch_size, activation, lr=1e-3,
                 norm=None, bn_momentum=0.9, norm_eps=1e-5, dropout=0.0):
        # He-like scaling for stable starts
        self.W = torch.randn(input_dim, out_dim) / (input_dim ** 0.5)
        self.b = torch.zeros(1, out_dim)
        self.activation = activation
        self.lr = lr
        self.batch_size = batch_size

        # Optional BatchNorm after linear pre-activation
        self.bn = BatchNorm1d(out_dim, eps=norm_eps, momentum=bn_momentum) if norm == 'batchnorm' else None
        self.ln = LayerNorm1d(out_dim, eps=norm_eps) if norm == 'layernorm' else None
        self.rmsn = RMSNorm(out_dim, eps=norm_eps) if norm == 'rmsnorm' else None
        self.dropout = Dropout(dropout) if dropout > 0 else None
        # Caches
        self.X, self.z, self.z_tilde, self.a = None, None, None, None
        self.dW, self.db = None, None

        self.parameter = {'name': 'Linear',
                          'size': [input_dim, out_dim],
                          'activation': activation,
                          'batchnorm': bool(self.bn is not None),
                          'layernorm': bool(self.ln is not None),
                          'rmsnorm': bool(self.rmsn is not None),
                          'dropout': bool(self.dropout is not None)}

    def _activation(self, z):
        if self.activation == 'relu':
            return torch.maximum(torch.zeros_like(z), z)
        elif self.activation == 'sigmoid':
            return 1 / (1 + torch.exp(-z))
        elif self.activation == 'tanh':
            return torch.tanh(z)
        else:
            return z

    def _activation_deriv(self, a):
        if self.activation == 'relu':
            return (a > 0).to(a.dtype)
        elif self.activation == 'sigmoid':
            return a * (1 - a)
        elif self.activation == 'tanh':
            return 1 - a**2
        else:
            return torch.ones_like(a)

    def set_training(self, mode: bool):
        if self.bn is not None:
            self.bn.training = mode

    def forward(self, X):
        self.X = X                                                 # (B, in_dim)
        self.z = X @ self.W + self.b                               # pre-activation (B, out_dim)
        if self.bn is not None:
            self.z_tilde = self.bn.forward(self.z)
        elif self.ln is not None:
            self.z_tilde = self.ln.forward(self.z)
        elif self.rmsn is not None:
            self.z_tilde = self.rmsn.forward(self.z)
        else:
            self.z_tilde = self.z
        self.a = self._activation(self.z_tilde)                    # post-activation
        out = self.dropout.forward(self.a) if self.dropout else self.a
        return out

    def backward(self, d_a):
        m = self.X.shape[0]                                        # actual batch size (may be < batch_size at last step)
        d_a = self.dropout.backward(d_a) if self.dropout else d_a
        d_z_tilde = d_a * self._activation_deriv(self.a)           # (B, out_dim)
        if self.bn is not None:
            d_z = self.bn.backward(d_z_tilde)
        elif self.ln is not None:
            d_z = self.ln.backward(d_z_tilde)
        elif self.rmsn is not None:
            d_z = self.rmsn.backward(d_z_tilde)
        else:
            d_z = d_z_tilde
        d_W = self.X.t() @ d_z / m                                 # (in_dim, out_dim)
        d_b = d_z.sum(dim=0, keepdim=True) / m                     # (1, out_dim)
        d_X = d_z @ self.W.t()                                     # (B, in_dim)

        self.dW, self.db = d_W, d_b
        return d_X


class SoftMax:
    def __init__(self):
        self.parameter = {'name': 'SoftMax'}
        self.y_pred = None

    def forward(self, X):
        # stable softmax
        X_shift = X - X.max(dim=1, keepdim=True).values
        exp = torch.exp(X_shift)
        partition = exp.sum(dim=1, keepdim=True)
        self.y_pred = exp / partition
        return self.y_pred

    def backward(self, y):
        # Assuming y is one-hot (B, C)
        return self.y_pred - y


class MLP:
    def __init__(self, input_dim, output_classes, hidden_sizes, hidden_activations,
                 batch_size, lr=1e-3, norm=None, bn_momentum=0.9, norm_eps=1e-5, dropout=0.0):
        self.lr = lr
        self.hidden_layers = []
        for i, hidden_size in enumerate(hidden_sizes):
            self.hidden_layers.append(
                LinearLayer(input_dim, hidden_size, batch_size, hidden_activations[i], lr=lr,
                            norm=norm, bn_momentum=bn_momentum, norm_eps=norm_eps, dropout=dropout)
            )
            input_dim = hidden_size

        # Output layer: no BN, no activation here (Softmax follows)
        self.output_layer = LinearLayer(input_dim, output_classes, batch_size, None, lr=lr,
                                        norm=None)
        self.softmax = SoftMax()

    def set_training(self, mode: bool):
        for layer in self.hidden_layers:
            layer.set_training(mode)
        self.output_layer.set_training(mode)

    def forward(self, X):
        for layer in self.hidden_layers:
            X = layer.forward(X)
        X = self.output_layer.forward(X)
        X = self.softmax.forward(X)
        return X

    def predict(self, X):
        # forward without cache; BN uses running stats (eval)
        out = X
        for layer in self.hidden_layers:
            z = out @ layer.W + layer.b
            if layer.bn is not None:
                # Use running stats in eval mode
                x_hat = (z - layer.bn.running_mean) / torch.sqrt(layer.bn.running_var + layer.bn.eps)
                z = layer.bn.gamma * x_hat + layer.bn.beta
            elif layer.ln is not None:
                ln_mean = z.mean(dim=1, keepdim=True)
                ln_var = z.var(dim=1, unbiased=False, keepdim=True)
                z = (z - ln_mean) / torch.sqrt(ln_var + layer.ln.eps) * layer.ln.gamma + layer.ln.beta

            elif layer.rmsn is not None:
                rms = torch.sqrt(z.pow(2).mean(dim=1, keepdim=True) + layer.rmsn.eps)
                z = layer.rmsn.gamma * (z / rms) + layer.rmsn.beta

            out = layer._activation(z)
        z = out @ self.output_layer.W + self.output_layer.b
        probs = self.softmax.forward(z)
        return probs

    def backward(self, y):
        d_a = self.softmax.backward(y)
        d_a = self.output_layer.backward(d_a)
        for layer in reversed(self.hidden_layers):
            d_a = layer.backward(d_a)
        return d_a

    def print_parameters(self):
        for i, layer in enumerate(self.hidden_layers + [self.output_layer]):
            print(f"Layer {i}: {layer.parameter}")
            if getattr(layer, 'bn', None) is not None:
                print(f"         BN: {layer.bn.parameter}")
            if getattr(layer, 'ln', None) is not None:
                print(f"         LayerNorm: {layer.ln.parameter}")
            elif getattr(layer, 'rmsn', None) is not None:
                print(f"         RMSNorm: {layer.rmsn.parameter}")
            if getattr(layer, 'dropout', None) is not None:
                print(f"         Dropout: {layer.dropout.parameter}")

    def params_dict(self):
        params = {}
        for i, layer in enumerate(self.hidden_layers):
            params[f"W_hidden_{i}"] = layer.W
            params[f"b_hidden_{i}"] = layer.b
            if layer.bn is not None:
                params[f"gamma_{i}"] = layer.bn.gamma
                params[f"beta_{i}"]  = layer.bn.beta
            if layer.ln is not None:
                params[f"gamma_{i}"] = layer.ln.gamma
                params[f"beta_{i}"]  = layer.ln.beta
            if layer.rmsn is not None:
                params[f"gamma_{i}"] = layer.rmsn.gamma
                params[f"beta_{i}"]  = layer.rmsn.beta
        params["W_out"] = self.output_layer.W
        params["b_out"] = self.output_layer.b
        return params

    def grads_dict(self):
        grads = {}
        for i, layer in enumerate(self.hidden_layers):
            grads[f"W_hidden_{i}"] = layer.dW
            grads[f"b_hidden_{i}"] = layer.db
            if layer.bn is not None:
                grads[f"gamma_{i}"] = layer.bn.d_gamma
                grads[f"beta_{i}"]  = layer.bn.d_beta
            if layer.ln is not None:
                grads[f"gamma_{i}"] = layer.ln.d_gamma
                grads[f"beta_{i}"]  = layer.ln.d_beta
            if layer.rmsn is not None:
                grads[f"gamma_{i}"] = layer.rmsn.d_gamma
                grads[f"beta_{i}"]  = layer.rmsn.d_beta
        grads["W_out"] = self.output_layer.dW
        grads["b_out"] = self.output_layer.db
        return grads

In [53]:
class SGD:
    def __init__(self, params, lr=1e-3):
        self.params = params  
        self.lr = lr

    def step(self, grads):
        # In-place SGD: param <- param - lr * grad
        for k in self.params.keys():
            g = grads.get(k, None)
            if g is None:
                continue
            self.params[k] -= self.lr * g

class AdamW:
    def __init__(self, params, lr=1e-3, beta1=0.9, beta2=0.999, eps=1e-8, weight_decay=0.01):
        self.params = params
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.weight_decay = weight_decay

        # First/second moments (same device/dtype as each param)
        self.m = {k: torch.zeros_like(v) for k, v in params.items()}
        self.v = {k: torch.zeros_like(v) for k, v in params.items()}
        self.t = 0  # time step

    def step(self, grads):
        self.t += 1
        b1, b2 = self.beta1, self.beta2
        # Bias correction terms (scalars)
        bc1 = 1.0 - (b1 ** self.t)
        bc2 = 1.0 - (b2 ** self.t)

        for k, p in self.params.items():
            g = grads.get(k, None)
            if g is None:
                continue

            self.m[k] = b1 * self.m[k] + (1.0 - b1) * g
            self.v[k] = b2 * self.v[k] + (1.0 - b2) * (g * g)

            # Bias correction
            m_hat = self.m[k] / bc1
            v_hat = self.v[k] / bc2

            # Decoupled weight decay
            if self.weight_decay != 0.0:
                p.add_(p, alpha=-self.lr * self.weight_decay)  # p <- p - lr*wd*p

            # Parameter update
            denom = torch.sqrt(v_hat) + self.eps
            p.addcdiv_(m_hat, denom, value=-self.lr)  # p <- p - lr * m_hat / denom


In [54]:
import torch
import torch.nn.functional as F
from sklearn.datasets import load_iris

def predict_classes(mlp, X):
    with torch.no_grad():
        return torch.argmax(mlp.predict(X), dim=1)

def onehot_to_index(Y):
    return torch.argmax(Y, dim=1)

torch.manual_seed(42)
iris = load_iris()
X = torch.tensor(iris.data, dtype=torch.float32)
y = torch.tensor(iris.target, dtype=torch.long)

N = X.shape[0]
perm = torch.randperm(N)
split = int(0.8 * N)
idx_train, idx_test = perm[:split], perm[split:]

X_train, X_test = X[idx_train], X[idx_test]
y_train_idx, y_test_idx = y[idx_train], y[idx_test]

C = int(y.max().item() + 1)
y_train = F.one_hot(y_train_idx, num_classes=C).float()
y_test = F.one_hot(y_test_idx, num_classes=C).float()

input_dim = X_train.shape[1]
output_classes = C
hidden_sizes = [32, 16]
hidden_activations = ['relu', 'relu']
batch_size = 32
lr = 0.01
epochs = 30
dropout = 0.3
norm = 'rmsnorm'

mlp = MLP(input_dim, output_classes, hidden_sizes, hidden_activations, batch_size, lr, norm=norm, dropout=dropout)
mlp.print_parameters()
opt = AdamW(mlp.params_dict(), lr=lr, weight_decay=0.01)

for epoch in range(epochs):
    for i in range(0, X_train.shape[0], batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        mlp.forward(X_batch)
        mlp.backward(y_batch)
        opt.step(mlp.grads_dict())
    if (epoch + 1) % 10 == 0:
        y_train_pred = predict_classes(mlp, X_train)
        y_test_pred = predict_classes(mlp, X_test)
        train_acc = (y_train_pred == y_train_idx).float().mean().item()
        test_acc = (y_test_pred == y_test_idx).float().mean().item()
        print(f"Epoch {epoch + 1}/{epochs}  Train Acc: {train_acc:.4f}  Test Acc: {test_acc:.4f}")


Layer 0: {'name': 'Linear', 'size': [4, 32], 'activation': 'relu', 'batchnorm': False, 'layernorm': False, 'rmsnorm': True, 'dropout': True}
         RMSNorm: {'name': 'RMSNorm', 'size': [1, 32]}
         Dropout: {'name': 'Dropout', 'p': 0.3}
Layer 1: {'name': 'Linear', 'size': [32, 16], 'activation': 'relu', 'batchnorm': False, 'layernorm': False, 'rmsnorm': True, 'dropout': True}
         RMSNorm: {'name': 'RMSNorm', 'size': [1, 16]}
         Dropout: {'name': 'Dropout', 'p': 0.3}
Layer 2: {'name': 'Linear', 'size': [16, 3], 'activation': None, 'batchnorm': False, 'layernorm': False, 'rmsnorm': False, 'dropout': False}
Epoch 10/30  Train Acc: 0.8333  Test Acc: 0.7667
Epoch 20/30  Train Acc: 0.9917  Test Acc: 0.9000
Epoch 30/30  Train Acc: 0.9833  Test Acc: 0.9000
